# Preparation

In [0]:
# Install pip libraries
!pip3 install lxml
!pip3 install html5lib

     |██████████

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 7.1 MB 6.7 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-4204c80a-7d60-4ab7-b9e2-22d73e34ef99/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 112 kB 4.8 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-4204c80a-7d60-4ab7-b9e2-22d73e34ef99/bin/python -m pip install --upgrade pip' command.


In [0]:
# Import libraries
import numpy as np
import pandas as pd
import pyspark as ps
import pyspark.pandas as pspa
from pyspark.sql import functions as F

# 1. Extraction

## 1.1. Import tables from URLs (as lists of pandas dataframes)

In [0]:
# Define source URLs
url_top_channels = 'http://us.youtubers.me/global/all/top-1000-youtube-channels'
url_top_videos = 'https://us.youtubers.me/global/all/top-1000-youtube-videos'
url_most_subscribed_channels = 'https://us.youtubers.me/global/all/top-1000-most-subscribed-youtube-channels'
url_most_popular_channels = 'https://us.youtubers.me/global/all/top-1000-most-popular-youtube-channels'
url_most_popular_videos = 'https://us.youtubers.me/global/all/top-1000-most-popular-youtube-videos'

# Import df lists from source URLs
dflist_top_channels = pd.read_html(io = url_top_channels)
dflist_top_videos = pd.read_html(io = url_top_videos)
dflist_most_subscribed_channels = pd.read_html(io = url_top_videos)
dflist_most_popular_channels = pd.read_html(io=url_most_popular_channels)
dflist_most_popular_videos = pd.read_html(io=url_most_popular_videos)

## 1.2. Extract dataframes from lists

Each dataframe list has 3 elements:
- Element 1: Dataframe 1
- Element 2: Dataframe 2 (part 1)
- Element 3: Dataframe 2 (part 2)

In order to create DF2, we must .concat() E2 and E3.

In [0]:
# Create dataframes from E1s
df_top_channels = dflist_top_channels[0]
df_top_videos = dflist_top_videos[0]
df_most_subscribed_channels = dflist_most_subscribed_channels[0]
df_most_popular_channels = dflist_most_popular_channels[0]
df_most_popular_videos = dflist_most_popular_videos[0]

# Rename E3s' column names to match E2s'
dflist_top_channels[2].columns = dflist_top_channels[1].columns.values.tolist()
dflist_top_videos[2].columns = dflist_top_videos[1].columns.values.tolist()
dflist_most_subscribed_channels[2].columns = dflist_most_subscribed_channels[1].columns.values.tolist()
dflist_most_popular_channels[2].columns = dflist_most_popular_channels[1].columns.values.tolist()
dflist_most_popular_videos[2].columns = dflist_most_popular_videos[1].columns.values.tolist()

# Create dataframes by concating E2s and E3s
df_top_channels_metrics = pd.concat(dflist_top_channels[1:])
df_top_videos_metrics = pd.concat(dflist_top_videos[1:])
df_most_subscribed_channels_metrics = pd.concat(dflist_most_subscribed_channels[1:])
df_most_popular_channels_metrics = pd.concat(dflist_most_popular_channels[1:])
df_most_popular_videos_metrics = pd.concat(dflist_most_popular_videos[1:])

## 1.3. Convert dataframes from pandas to spark

In [0]:
# Convert created pandas dataframes to spark dataframes
df_top_channels = spark.createDataFrame(df_top_channels)
df_top_channels_metrics = spark.createDataFrame(df_top_channels_metrics)

df_top_videos = spark.createDataFrame(df_top_videos)
df_top_videos_metrics = spark.createDataFrame(df_top_videos_metrics)

df_most_subscribed_channels = spark.createDataFrame(df_most_subscribed_channels)
df_most_subscribed_channels_metrics = spark.createDataFrame(df_most_subscribed_channels_metrics)

df_most_popular_channels = spark.createDataFrame(df_most_popular_channels)
df_most_popular_channels_metrics = spark.createDataFrame(df_most_popular_channels_metrics)

df_most_popular_videos = spark.createDataFrame(df_most_popular_videos)
df_most_popular_videos_metrics = spark.createDataFrame(df_most_popular_videos_metrics)

## 1.4. Display imported, raw dataframes

In [0]:
# Display all imported, raw dataframes
print('\nTop Channels')
display(df_top_channels)

print('\nTop Channels (metrics)')
display(df_top_channels_metrics)

print('\nTop Videos')
display(df_top_videos)

print('\nTop Videos (metrics)')
display(df_top_videos_metrics)

print('\nMost Subscribed Channels')
display(df_most_subscribed_channels)

print('\nMost Subscribed Channels (metrics)')
display(df_most_subscribed_channels_metrics)

print('\nMost Popular Channels')
display(df_most_popular_channels)

print('\nMost Popular Channels (metrics)')
display(df_most_popular_channels_metrics)

print('\nMost Popular Videos')
display(df_most_popular_videos)

print('\nMost Popular Videos (metrics)')
display(df_most_popular_videos_metrics)


Top Channels


rank,Youtuber,subscribers,video views,video count,category,started
1,T-Series,236000000,215448719927,18742,Music,2006
2,Cocomelon - Nursery Rhymes,154000000,151495855157,857,Education,2006
3,SET India,151000000,139508515842,104998,Shows,2006
4,Sony SAB,77200000,92411723276,64568,Shows,2007
5,✿ Kids Diana Show,108000000,88008748808,1065,People & Blogs,2015
6,Like Nastya,104000000,87845862611,760,People & Blogs,2016
7,WWE,93400000,74213926197,66653,Sports,2007
8,Vlad and Niki,93400000,72957816965,527,Entertainment,2018
9,Movieclips,58700000,59058009915,39988,Film & Animation,2006
10,Colors TV,59900000,57721177728,103569,Shows,2008



Top Channels (metrics)


rank,Youtuber,subscribers/year,Video views/Year,Video count/Year
1,T-Series,13882352,12673454113,1102
2,Cocomelon - Nursery Rhymes,9058823,8911520891,50
3,SET India,8882352,8206383284,6176
4,Sony SAB,4825000,5775732704,4035
5,✿ Kids Diana Show,13500000,11001093601,133
6,Like Nastya,14857142,12549408944,108
7,WWE,5837500,4638370387,4165
8,Vlad and Niki,18680000,14591563393,105
9,Movieclips,3452941,3474000583,2352
10,Colors TV,3993333,3848078515,6904



Top Videos


...

rank,Youtuber,subscribers/year,Video views/Year,Video count/Year
1,AnnaKendrickVEVO,143000,64513490,0
2,jawed,185000,14241667,0
3,icanrockyourworld,595625,318212846,0
4,dollarbill,145294,67949691,0
5,Wisin & Yandel,1418000,1201873035,0
6,Wiz Khalifa Music,6530,6122690509,2
7,RahatFAKhanVEVO,98571,103790405,0
8,4NonBlondesVEVO,107142,106965908,0
9,CALIDANDEE,82250,44109541,0
10,DrDreVEVO,235714,202120993,0



Most Popular Videos


rank,Video,Video views,Likes,Dislikes,Category,published
1,Lil Nas X - Old Town Road (Official Movie) ft. Billy Ray Cyrus,54071677,3497955,78799.0,Music,2019
2,"20 Tennis shots if they were not filmed, NOBODY would believe them",3471237,19023,859.0,null,2017
3,Wiz Khalifa - See You Again ft. Charlie Puth [Official Video] Furious 7 Soundtrack,5786426309,40230004,null,Music,2015
4,JP Saxe - If the World Was Ending (Official Video) ft. Julia Michaels,76834495,804353,21195.0,Music,2019
5,"Polo G, Stunna 4 Vegas & NLE Choppa feat. Mike WiLL Made-It - Go Stupid (Official Video)",7396199,320910,6485.0,Music,2020
6,JD Pantoja - 12•19 (Official Video),7010732,1027392,34185.0,Music,2020
7,Power Star Pawan Kalyan Special Surprise To Sensational Singer Baby|Filmy Poster,96686,1007,82.0,Entertainment,2018
8,Rels B - LA LATINA (Videoclip Oficial),5851411,155543,3504.0,Music,2019
9,"Colapesce, Dimartino - Musica leggerissima (Official Video - Sanremo 2021)",6953838,116350,2059.0,Music,2021
10,"6ix9ine - FEFE ft. Nicki Minaj, Murda Beatz",1032248869,8721050,null,People & Blogs,2018



Most Popular Videos (metrics)


rank,Video,video views/year,likes/year,dislikes/year
1,Lil Nas X - Old Town Road (Official Movie) ft. Billy Ray Cyrus,13517919,874488,19699.0
2,"20 Tennis shots if they were not filmed, NOBODY would believe them",578539,3170,143.0
3,Wiz Khalifa - See You Again ft. Charlie Puth [Official Video] Furious 7 Soundtrack,723303288,5028750,0.0
4,JP Saxe - If the World Was Ending (Official Video) ft. Julia Michaels,19208623,201088,5298.0
5,"Polo G, Stunna 4 Vegas & NLE Choppa feat. Mike WiLL Made-It - Go Stupid (Official Video)",2465399,106970,2161.0
6,JD Pantoja - 12•19 (Official Video),2336910,342464,11395.0
7,Power Star Pawan Kalyan Special Surprise To Sensational Singer Baby|Filmy Poster,19337,201,16.0
8,Rels B - LA LATINA (Videoclip Oficial),1462852,38885,876.0
9,"Colapesce, Dimartino - Musica leggerissima (Official Video - Sanremo 2021)",3476919,58175,1029.0
10,"6ix9ine - FEFE ft. Nicki Minaj, Murda Beatz",206449773,1744210,0.0


## 1.5. Save raw dataframes as a parquet files (1st layer)

In [0]:
# Delete raw data from previous runs
dbutils.fs.rm('dbfs:/FileStore/raw', True)

# Save dataframes as parquet files
df_top_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topChannels')

df_top_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topChannelsMetrics')

df_top_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topVideos')

df_top_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/topVideosMetrics')

df_most_subscribed_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostSubscribed')

df_most_subscribed_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostSubscribedMetrics')

df_most_popular_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularChannels')

df_most_popular_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularChannelsMetrics')

df_most_popular_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularVideos')

df_most_popular_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/raw/mostPopularVideosMetrics')

# 2. Transformation

## 2.1. Top Channels

### Clean column headers

In [0]:
# Show dataframe column names and types
df_top_channels.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_top_channels.columns]

# Create dataframe with new column headers
df_top_channels = df_top_channels.toDF(*newColumns)

# Show dataframe with updated column names
df_top_channels.printSchema()

root
 |-- rank: long (nullable = true)
 |-- Youtuber: string (nullable = true)
 |-- subscribers: long (nullable = true)
 |-- video views: long (nullable = true)
 |-- video count: long (nullable = true)
 |-- category: string (nullable = true)
 |-- started: long (nullable = true)

root
 |-- rank: long (nullable = true)
 |-- youtuber: string (nullable = true)
 |-- subscribers: long (nullable = true)
 |-- video_views: long (nullable = true)
 |-- video_count: long (nullable = true)
 |-- category: string (nullable = true)
 |-- started: long (nullable = true)



### Clean column values

In [0]:
# Show dataframe values
display(df_top_channels)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_top_channels = df_top_channels\
    .withColumn('youtuber', F.trim(df_top_channels['youtuber']))\
    .withColumn('youtuber', F.lower(F.col('youtuber')))\
    .withColumn('subscribers', F.translate('subscribers', ',', ''))\
    .withColumn('video_views', F.translate('video_views', ',', ''))\
    .withColumn('video_count', F.translate('video_count', ',', ''))\
    .withColumn('category', F.trim(df_top_channels['category']))\
    .withColumn('category', F.lower(F.col('category')))

# Show dataframe with updated values
display(df_top_channels)

rank,youtuber,subscribers,video_views,video_count,category,started
1,T-Series,236000000,215448719927,18742,Music,2006
2,Cocomelon - Nursery Rhymes,154000000,151495855157,857,Education,2006
3,SET India,151000000,139508515842,104998,Shows,2006
4,Sony SAB,77200000,92411723276,64568,Shows,2007
5,✿ Kids Diana Show,108000000,88008748808,1065,People & Blogs,2015
6,Like Nastya,104000000,87845862611,760,People & Blogs,2016
7,WWE,93400000,74213926197,66653,Sports,2007
8,Vlad and Niki,93400000,72957816965,527,Entertainment,2018
9,Movieclips,58700000,59058009915,39988,Film & Animation,2006
10,Colors TV,59900000,57721177728,103569,Shows,2008


rank,youtuber,subscribers,video_views,video_count,category,started
1,t-series,236000000,215448719927,18742,music,2006
2,cocomelon - nursery rhymes,154000000,151495855157,857,education,2006
3,set india,151000000,139508515842,104998,shows,2006
4,sony sab,77200000,92411723276,64568,shows,2007
5,✿ kids diana show,108000000,88008748808,1065,people & blogs,2015
6,like nastya,104000000,87845862611,760,people & blogs,2016
7,wwe,93400000,74213926197,66653,sports,2007
8,vlad and niki,93400000,72957816965,527,entertainment,2018
9,movieclips,58700000,59058009915,39988,film & animation,2006
10,colors tv,59900000,57721177728,103569,shows,2008


### Update data types

In [0]:
# Show dataframe column names and types
df_top_channels.printSchema()

root
 |-- rank: long (nullable = true)
 |-- youtuber: string (nullable = true)
 |-- subscribers: string (nullable = true)
 |-- video_views: string (nullable = true)
 |-- video_count: string (nullable = true)
 |-- category: string (nullable = true)
 |-- started: long (nullable = true)



In [0]:
# Show dataframe column names and types
df_top_channels.printSchema()

# Create dataframe with updated data types
df_top_channels = df_top_channels\
    .withColumn('rank', F.col('rank').cast('int'))\
    .withColumn('subscribers', F.col('subscribers').cast('int'))\
    .withColumn('video_views', F.col('video_views').cast('long'))\
    .withColumn('video_count', F.col('video_count').cast('int'))\
    .withColumn('started', F.col('started').cast('int'))

# Show dataframe with updated data types
df_top_channels.printSchema()

root
 |-- rank: long (nullable = true)
 |-- youtuber: string (nullable = true)
 |-- subscribers: string (nullable = true)
 |-- video_views: string (nullable = true)
 |-- video_count: string (nullable = true)
 |-- category: string (nullable = true)
 |-- started: long (nullable = true)

root
 |-- rank: integer (nullable = true)
 |-- youtuber: string (nullable = true)
 |-- subscribers: integer (nullable = true)
 |-- video_views: long (nullable = true)
 |-- video_count: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- started: integer (nullable = true)



## 2.2. Top Videos

### Clean column headers

In [0]:
# Show dataframe column names and types
df_top_videos.printSchema()

# Create list with stripped, lowercased, snake-cased column headers
newColumns = [column.strip()\
                    .lower()\
                    .replace(' ', '_')
              for column
              in df_top_videos.columns]

# Create dataframe with new column headers
df_top_videos = df_top_videos.toDF(*newColumns)

# Show dataframe with updated column names
df_top_videos.printSchema()

root
 |-- rank: long (nullable = true)
 |-- Video: string (nullable = true)
 |-- Video views: long (nullable = true)
 |-- Likes: double (nullable = true)
 |-- Dislikes: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- published: long (nullable = true)

root
 |-- rank: long (nullable = true)
 |-- video: string (nullable = true)
 |-- video_views: long (nullable = true)
 |-- likes: double (nullable = true)
 |-- dislikes: double (nullable = true)
 |-- category: string (nullable = true)
 |-- published: long (nullable = true)



### Clean column values

In [0]:
# Show dataframe values
display(df_top_videos)

# Create dataframe with clean values (trim left/right whitespace, lowercase STRs, remove commas from numbers)
df_top_videos = df_top_videos\
    .withColumn('video', F.trim(df_top_videos['video']))\
    .withColumn('video', F.lower(F.col('video')))\
    .withColumn('category', F.trim(df_top_videos['category']))\
    .withColumn('category', F.lower(F.col('category')))

# Show dataframe with updated values
display(df_top_videos)

rank,video,video_views,likes,dislikes,category,published
1,Baby Shark Dance | #babyshark Most Viewed Video | Animal Songs | PINKFONG Songs for Children,12289336029,3.9254972E7,null,Education,2016
2,Luis Fonsi - Despacito ft. Daddy Yankee,8085851950,5.0839026E7,null,Music,2017
3,Johny Johny Yes Papa 👶 THE BEST Song for Children | LooLoo Kids,6617395127,1.8917192E7,null,Music,2016
4,Bath Song | CoComelon Nursery Rhymes & Kids Songs,6013771759,1.6000932E7,null,Education,2018
5,Ed Sheeran - Shape of You (Official Music Video),5915389177,3.1086459E7,null,Music,2017
6,Wiz Khalifa - See You Again ft. Charlie Puth [Official Video] Furious 7 Soundtrack,5781504867,4.0196749E7,null,Music,2015
7,Phonics Song with TWO Words - A For Apple - ABC Alphabet Songs with Sounds for Children,5158156126,null,null,Education,2014
8,Wheels on the Bus | CoComelon Nursery Rhymes & Kids Songs,4925305477,1.4458336E7,null,Education,2018
9,Mark Ronson - Uptown Funk (Official Video) ft. Bruno Mars,4828795409,2.0097266E7,null,Music,2014
10,Учим цвета Разноцветные яйца на ферме Miroshka Tv,4814183676,1.346698E7,null,Education,2018


rank,video,video_views,likes,dislikes,category,published
1,baby shark dance | #babyshark most viewed video | animal songs | pinkfong songs for children,12289336029,3.9254972E7,null,education,2016
2,luis fonsi - despacito ft. daddy yankee,8085851950,5.0839026E7,null,music,2017
3,johny johny yes papa 👶 the best song for children | looloo kids,6617395127,1.8917192E7,null,music,2016
4,bath song | cocomelon nursery rhymes & kids songs,6013771759,1.6000932E7,null,education,2018
5,ed sheeran - shape of you (official music video),5915389177,3.1086459E7,null,music,2017
6,wiz khalifa - see you again ft. charlie puth [official video] furious 7 soundtrack,5781504867,4.0196749E7,null,music,2015
7,phonics song with two words - a for apple - abc alphabet songs with sounds for children,5158156126,null,null,education,2014
8,wheels on the bus | cocomelon nursery rhymes & kids songs,4925305477,1.4458336E7,null,education,2018
9,mark ronson - uptown funk (official video) ft. bruno mars,4828795409,2.0097266E7,null,music,2014
10,учим цвета разноцветные яйца на ферме miroshka tv,4814183676,1.346698E7,null,education,2018


### Update data types

In [0]:
# Show dataframe column names and types
df_top_videos.printSchema()

root
 |-- rank: long (nullable = true)
 |-- video: string (nullable = true)
 |-- video_views: long (nullable = true)
 |-- likes: double (nullable = true)
 |-- dislikes: double (nullable = true)
 |-- category: string (nullable = true)
 |-- published: long (nullable = true)



In [0]:
# Show dataframe column names and types
df_top_videos.printSchema()

# Create dataframe with updated data types
df_top_videos = df_top_videos\
    .withColumn('video_views', F.col('video_views').cast('long'))\
    .withColumn('likes', F.col('likes').cast('long'))\
    .withColumn('published', F.col('published').cast('int'))

# Show dataframe with updated data types
df_top_videos.printSchema()

root
 |-- rank: long (nullable = true)
 |-- video: string (nullable = true)
 |-- video_views: long (nullable = true)
 |-- likes: double (nullable = true)
 |-- dislikes: double (nullable = true)
 |-- category: string (nullable = true)
 |-- published: long (nullable = true)

root
 |-- rank: long (nullable = true)
 |-- video: string (nullable = true)
 |-- video_views: long (nullable = true)
 |-- likes: long (nullable = true)
 |-- dislikes: double (nullable = true)
 |-- category: string (nullable = true)
 |-- published: integer (nullable = true)



## 2.3. Top Channels (metrics)

## 2.4. Top Videos (metrics)

## 2.5. Most Subscribed Channels

## 2.6. Most Subscribed Channels (metrics)

## 2.7. Most Popular Channels

## 2.8. Most Popular Channels (metrics)

## 2.9. Most Popular Videos

## 2.10. Most Popular Videos (metrics)

## 2.11. Save refined dataframes as parquet files (2nd layer)

In [0]:
# Delete refined data from previous runs
dbutils.fs.rm('dbfs:/FileStore/refined', True)

# Save dataframes as parquet files
df_top_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topChannels')

df_top_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topChannelsMetrics')

df_top_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topVideos')

df_top_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/topVideosMetrics')

df_most_subscribed_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostSubscribed')

df_most_subscribed_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostSubscribedMetrics')

df_most_popular_channels\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularChannels')

df_most_popular_channels_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularChannelsMetrics')

df_most_popular_videos\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularVideos')

df_most_popular_videos_metrics\
    .write\
    .format('parquet')\
    .option('header', 'true')\
    .save('dbfs:/FileStore/refined/mostPopularVideosMetrics')